<a href="https://colab.research.google.com/github/ms236566/-/blob/main/Kr_TO_DPI23_1_MoskvichevaOV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Контрольная работа

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re
import csv
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
with open('/content/drive/MyDrive/KRTO/preprocessed_descriptions.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    unique_words = set()
    for row in reader:
        text = row[-1]
        words = word_tokenize(text)

        for word in words:
            unique_words.add(word)

print(unique_words)

{'voodoo', 'ancienti', 'reezohghahloh', 'greatlooking', 'supped', 'wilbur', 'lazyme', 'europeanstyle', 'podracky', 'variants', 'form', 'moons', 'rogene', 'sause', 'cm', 'personally', 'seco', 'roussos', 'tortelloni', 'sheetthey', 'lagers', 'overmix', 'dude', 'dada', 'iw', 'tulip', 'ree', 'sapna', 'httpwwwdiabetesorgcommunityprogramsandlocaleventsdiabetescampsiahollowjsp', 'easilygreat', 'fresco', 'auriga', 'kroepoek', 'wolcott', 'couriers', 'cracks', 'shorties', 'police', 'hilton', 'scungilli', 'supermarketthis', 'elmotoo', 'fugdebut', 'wwwilovecheesecom', 'hair', 'pirates', 'accentuates', 'avid', 'recieve', 'arum', 'afteraharddayattheoffice', 'squishiness', 'nana', 'noticeable', 'buttershot', 'betterthanstarbucks', 'magrey', 'reports', 'dug', 'shook', 'texturethey', 'lasting', 'bummerby', 'cranberriesi', 'mirjs', 'picturesque', 'shreddingmincing', 'squeezingits', 'pichet', 'stawberry', 'haricots', 'market', 'steakto', 'wk', 'guestsserve', 'chef1ab', 'devil', 'yumminess', 'barnardthis',

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [4]:
import random
import editdistance

unique_words_list = list(unique_words)
for _ in range(5):
    word1, word2 = random.sample(unique_words_list, 2)
    distance = editdistance.eval(word1, word2)
    print(f"1: {word1:^10}, 2: {word2:^12}, Расстояние редактирования между 1 и 2: {distance}")

1: hootenanny, 2:   3degree   , Расстояние редактирования между 1 и 2: 9
1:   towns   , 2:   scottish  , Расстояние редактирования между 1 и 2: 6
1: macedonia , 2: spainrecipescom, Расстояние редактирования между 1 и 2: 13
1: morethanks, 2:     err     , Расстояние редактирования между 1 и 2: 9
1: cheeserecipe, 2:     shes    , Расстояние редактирования между 1 и 2: 9


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [7]:
def closest_words(word, unique_words, k):
  distances = {}

  for w in unique_words_list:
    distance = editdistance.eval(word, w)
    distances[w] = distance

  closest = sorted(distances, key=distances.get)[:k]
  return closest

word = random.choice(unique_words_list)
k = int(input('Введите количество значений "k":'))

closest = closest_words(word, words, k)
print(f"Для слова:'{word}'ближайшие слова из списка: ({closest}")

Введите количество значений "k":5
Для слова:'perfumes'ближайшие слова из списка: (['perfumes', 'perfumed', 'fumes', 'perdue', 'jeromes']


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами:
    * word
    * stemmed_word
    * normalized_word

Столбец `word` укажите в качестве индекса.

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [8]:
import nltk
nltk.download('wordnet')
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [9]:
df = pd.DataFrame(unique_words, columns=['word'])

stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

df['stemmed_word'] = df['word'].apply(lambda x: stemmer.stem(x))
df['normalized_word'] = df['word'].apply(lambda x: lemmatizer.lemmatize(x))
df = df.set_index('word')
df.head(5)

,stemmed_word,normalized_word
word,,
voodoo,voodoo,voodoo
ancienti,ancienti,ancienti
reezohghahloh,reezohghahloh,reezohghahloh
greatlooking,greatlook,greatlooking
supped,sup,supped


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

> Добавить блок с цитатой



In [58]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
df = pd.read_csv('/content/drive/MyDrive/KRTO/preprocessed_descriptions.csv')
df['description'] = df['description'].astype(str)

stop = stopwords.words('english')
pos_tweets = df['description']
test = pd.DataFrame(pos_tweets)
test.columns = ["original"]
test['description_without_stopwords'] = test['original'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test

,original,description_without_stopwords
0,an original recipe created by chef scott meska...,"original recipe created chef scott meskan, geo..."
1,my children and their friends ask for my homem...,children friends ask homemade popsicles mornin...
2,"these were so go, it surprised even me.","go, surprised even me."
3,my sister-in-law made these for us at a family...,sister-in-law made us family get together. del...
4,i think a fondue is a very romantic casual din...,think fondue romantic casual dinner wonderful ...
...,...,...
29995,this is based on a french recipe but i changed...,based french recipe changed substantially. war...
29996,"this is a traditional fresh plum cake, thought...","traditional fresh plum cake, thought originate..."
29997,this is a traditional late summer early fall s...,"traditional late summer early fall snack, usua..."
29998,this is a delicious soup that i originally fou...,delicious soup originally found better homes g...


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
import random

df = pd.read_csv('/content/drive/MyDrive/KRTO/preprocessed_descriptions.csv')

random_recipes = df.sample(n=5)

tfidf_vectorizer = TfidfVectorizer()

for index, row in random_recipes.iterrows():
    description_vector = tfidf_vectorizer.fit_transform([row['description']])
    print(f"Рецепт {index + 1}:\n{description_vector}")

Рецепт 28781:
  (0, 8)	0.15249857033260467
  (0, 7)	0.15249857033260467
  (0, 19)	0.15249857033260467
  (0, 13)	0.15249857033260467
  (0, 14)	0.15249857033260467
  (0, 27)	0.15249857033260467
  (0, 24)	0.15249857033260467
  (0, 20)	0.15249857033260467
  (0, 1)	0.15249857033260467
  (0, 0)	0.15249857033260467
  (0, 2)	0.30499714066520933
  (0, 28)	0.15249857033260467
  (0, 3)	0.15249857033260467
  (0, 6)	0.15249857033260467
  (0, 12)	0.15249857033260467
  (0, 9)	0.15249857033260467
  (0, 21)	0.15249857033260467
  (0, 4)	0.15249857033260467
  (0, 25)	0.15249857033260467
  (0, 29)	0.15249857033260467
  (0, 23)	0.15249857033260467
  (0, 18)	0.15249857033260467
  (0, 17)	0.15249857033260467
  (0, 5)	0.30499714066520933
  (0, 16)	0.30499714066520933
  (0, 10)	0.15249857033260467
  (0, 15)	0.15249857033260467
  (0, 30)	0.15249857033260467
  (0, 22)	0.15249857033260467
  (0, 26)	0.30499714066520933
  (0, 11)	0.15249857033260467
Рецепт 11074:
  (0, 8)	0.1
  (0, 13)	0.1
  (0, 4)	0.1
  (0, 27)	0.

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).